In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time


In [2]:
start_day, end_day = 1462, 10650
df_data = pd.read_pickle(f"/srv/scratch/z5297792/Chapter2/df_data_{start_day}_{end_day}.pkl")
df_data = df_data.dropna().copy() # If ESPRA didnt work then we dont have vorticity, which is required for tracking

df_data = df_data[(df_data['Day']>=start_day)&(df_data['Day']<=end_day)]

In [3]:
# def tracking(df_data, VORT_WEIGHT=15000, R_THRESH=30):

#     df_data = df_data.copy()
#     # Initialize IDs: set first day IDs equal to their Eddy values.
#     min_day = df_data['Day'].min()
#     df_data['ID'] = -1
#     df_data.loc[df_data['Day'] == min_day, 'ID'] = np.arange(len(df_data.loc[df_data['Day'] == min_day, 'ID'])) # for the initial dataset
#     df_data['ID'] = df_data['ID'].astype('Int64')
#     next_num = df_data['ID'].max() + 1

#     R_df = pd.DataFrame(columns=['D_diff', 'W_diff', 'R'])

#     # Loop through days starting from min_day+1 to max_day.
#     for day in range(min_day + 1, df_data['Day'].max() + 1):
#         pres_day = df_data[df_data['Day'] == day].copy()
#         pres_day = pres_day[~np.isnan(pres_day['x0'])]
#         for e_pres in pres_day['Eddy'].unique():
#             pres_eddy = pres_day[pres_day['Eddy'] == e_pres].iloc[0]
#             assigned = False
#             # Look back up to 4 days.
#             for delta in range(1, 5):
#                 candidate_day = day - delta
#                 if candidate_day < 0:
#                     continue
#                 candidate_prev = df_data[df_data['Day'] == candidate_day].copy()
#                 candidate_prev = candidate_prev[~np.isnan(candidate_prev['x0'])]
#                 for e_prev in candidate_prev['Eddy'].unique():
#                     prev_eddy = candidate_prev[candidate_prev['Eddy'] == e_prev].iloc[0]
#                     R = np.sqrt(
#                         (pres_eddy['x0'] - prev_eddy['x0'])**2 +
#                         (pres_eddy['y0'] - prev_eddy['y0'])**2 +
#                         VORT_WEIGHT * (pres_eddy['w'] - prev_eddy['w'])**2    # IF ESPRA DIDNT WORK USE NENCIOLI!!
#                     )
#                     D_diff = np.hypot(pres_eddy['x0'] - prev_eddy['x0'], pres_eddy['y0'] - prev_eddy['y0'])
#                     W_diff = np.abs(pres_eddy['w'] - prev_eddy['w'])
#                     R_df.loc[len(R_df)] = {'D_diff': D_diff, 'W_diff': W_diff, 'R': R}
#                     if R < R_THRESH and (pres_eddy['Cyc'] == prev_eddy['Cyc']) and not (pres_day['ID'] == prev_eddy['ID']).any():
#                         df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = prev_eddy['ID']
#                         assigned = True
#                         break
#                 if assigned:
#                     break
#             if not assigned:
#                 df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = next_num
#                 next_num += 1

#         if day % 20 == 0:
#             print(day)

#     df_data['next_num'] = next_num
                
#     return df_data, R_df

# tic = time.perf_counter()

# df_eddies, R_df = tracking(df_data)

# toc = time.perf_counter()
# print(f"Elapsed time: {toc - tic:.4f} seconds")

In [4]:
# def tracking(df_data, start_ID, next_num, VORT_WEIGHT=15000, R_THRESH=30):

#     tic = time.perf_counter()

#     # Work on a copy so as not to modify the original DataFrame.
#     df_data = df_data.copy()
    
#     # Initialize IDs: set first day IDs equal to start_ID.
#     min_day = df_data['Day'].min()
#     df_data['ID'] = -1
#     df_data.loc[df_data['Day'] == min_day, 'ID'] = start_ID  # for the initial dataset
#     df_data['ID'] = df_data['ID'].astype('Int64')

#     # Precompute a dictionary mapping each day to the indices of rows where 'x0' is not NaN.
#     unique_days = sorted(df_data['Day'].unique())
#     day_dict = {
#         d: df_data.index[(df_data['Day'] == d) & (~df_data['x0'].isna())]
#         for d in unique_days
#     }

#     # Use a list to collect R values rather than appending row-by-row to a DataFrame.
#     df_R_rows = []
    
#     max_day = df_data['Day'].max()
#     for day in range(min_day + 1, max_day + 1):
#         # Skip this day if no valid rows exist.
#         if day not in day_dict or len(day_dict[day]) == 0:
#             continue

#         # Get all rows for the current day with valid x0.
#         pres_day = df_data.loc[day_dict[day]]
#         unique_eddies = pres_day['Eddy'].unique()

#         for e_pres in unique_eddies:
#             # Get the first occurrence for this eddy.
#             pres_eddy = pres_day[pres_day['Eddy'] == e_pres].iloc[0]
#             assigned = False

#             # Look back up to 4 days.
#             for delta in range(1, 5):
#                 candidate_day = day - delta
#                 if candidate_day < min_day:
#                     continue
#                 if candidate_day not in day_dict or len(day_dict[candidate_day]) == 0:
#                     continue

#                 # Get candidate previous day rows.
#                 candidate_prev = df_data.loc[day_dict[candidate_day]]
#                 # Group by 'Eddy' and take the first row for each unique eddy while preserving original order.
#                 candidate_group = candidate_prev.groupby('Eddy', as_index=False, sort=False).first()

#                 for _, prev_eddy in candidate_group.iterrows():
#                     # Compute differences and R value.
#                     dx = pres_eddy['x0'] - prev_eddy['x0']
#                     dy = pres_eddy['y0'] - prev_eddy['y0']
#                     dw = pres_eddy['w'] - prev_eddy['w']
#                     R = np.sqrt(dx ** 2 + dy ** 2 + VORT_WEIGHT * (dw ** 2))
#                     D_diff = np.hypot(dx, dy)
#                     W_diff = np.abs(dw)
#                     df_R_rows.append({'D_diff': D_diff, 'W_diff': W_diff, 'R': R})

#                     # Check if current candidate meets the criteria.
#                     if (R < R_THRESH) and (pres_eddy['Cyc'] == prev_eddy['Cyc']) and not (pres_day['ID'] == prev_eddy['ID']).any():
#                         df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = prev_eddy['ID']
#                         assigned = True
#                         break

#                 if assigned:
#                     break

#             if not assigned:
#                 df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = next_num
#                 next_num += 1

#         # Periodically print progress every 10 days.
#         if day % 200 == 0:
#             toc = time.perf_counter()
#             print(f"Elapsed time: {toc - tic:.4f} seconds")
#             print(day)

#     # Convert the accumulated R values list to a DataFrame.
#     df_R = pd.DataFrame(df_R_rows)
#     df_data['next_num'] = next_num

#     return df_data, df_R


In [5]:
def tracking(df_data, start_ID, next_num, VORT_WEIGHT=15000, R_THRESH=50):

    tic = time.perf_counter()
    df_data = df_data.copy()
    min_day = df_data['Day'].min()
    df_data['ID'] = -1
    df_data.loc[df_data['Day'] == min_day, 'ID'] = start_ID
    df_data['ID'] = df_data['ID'].astype('Int64')

    unique_days = sorted(df_data['Day'].unique())
    day_dict = {
        d: df_data.index[(df_data['Day'] == d) & (~df_data['x0'].isna())]
        for d in unique_days
    }

    df_R_rows = []
    max_day = df_data['Day'].max()

    for day in range(min_day + 1, max_day + 1):
        if day not in day_dict or len(day_dict[day]) == 0:
            continue

        pres_day = df_data.loc[day_dict[day]].copy()
        unique_eddies = pres_day['Eddy'].unique()
        assigned_IDs = set()  ### MODIFIED ### track assigned IDs per day

        for e_pres in unique_eddies:
            pres_eddy = pres_day[pres_day['Eddy'] == e_pres].iloc[0]
            best_match = None
            best_R = np.inf

            for delta in range(1, 5):
                candidate_day = day - delta
                if candidate_day < min_day:
                    continue
                if candidate_day not in day_dict or len(day_dict[candidate_day]) == 0:
                    continue

                candidate_prev = df_data.loc[day_dict[candidate_day]]
                candidate_group = candidate_prev.groupby('Eddy', as_index=False, sort=False).first()

                for _, prev_eddy in candidate_group.iterrows():
                    dx = pres_eddy['x0'] - prev_eddy['x0']
                    dy = pres_eddy['y0'] - prev_eddy['y0']
                    dw = pres_eddy['w'] - prev_eddy['w']
                    R = np.sqrt(dx ** 2 + dy ** 2 + VORT_WEIGHT * (dw ** 2))
                    D_diff = np.hypot(dx, dy)
                    W_diff = np.abs(dw)

                    df_R_rows.append({'D_diff': D_diff, 'W_diff': W_diff, 'R': R})

                    # Check match criteria
                    if (
                        R < R_THRESH
                        and pres_eddy['Cyc'] == prev_eddy['Cyc']
                        and prev_eddy['ID'] not in assigned_IDs  ### MODIFIED ###
                        and not pd.isna(prev_eddy['ID'])
                    ):
                        if R < best_R:
                            best_match = prev_eddy
                            best_R = R

            if best_match is not None:
                matched_id = best_match['ID']
                df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = matched_id
                assigned_IDs.add(matched_id)  ### MODIFIED ###
            else:
                df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = next_num
                assigned_IDs.add(next_num)  ### MODIFIED ###
                next_num += 1

        if day % 200 == 0:
            toc = time.perf_counter()
            print(f"Elapsed time: {toc - tic:.4f} seconds")
            print(day)

    df_R = pd.DataFrame(df_R_rows)
    df_data['next_num'] = next_num

    # Final check for duplicates
    assert not df_data.duplicated(subset=['ID', 'Day']).any(), "Duplicate (ID, Day) pairs found!"

    return df_data, df_R


In [6]:
start_ID = np.arange(len(df_data[df_data['Day']==df_data['Day'].min()]))
next_num = np.max(start_ID) + 1

df_eddies, df_R = tracking(df_data, start_ID, next_num)


Elapsed time: 83.7444 seconds
1600
Elapsed time: 273.4623 seconds
1800
Elapsed time: 394.5894 seconds
2000
Elapsed time: 546.0512 seconds
2200
Elapsed time: 665.3679 seconds
2400
Elapsed time: 793.1496 seconds
2600
Elapsed time: 930.6238 seconds
2800
Elapsed time: 1045.6990 seconds
3000
Elapsed time: 1171.1432 seconds
3200
Elapsed time: 1267.7612 seconds
3400
Elapsed time: 1406.7338 seconds
3600
Elapsed time: 1515.1264 seconds
3800
Elapsed time: 1673.9326 seconds
4000
Elapsed time: 1795.2377 seconds
4200
Elapsed time: 1941.3291 seconds
4400
Elapsed time: 2062.5138 seconds
4600
Elapsed time: 2202.2407 seconds
4800
Elapsed time: 2356.0831 seconds
5000
Elapsed time: 2495.6272 seconds
5200
Elapsed time: 2661.6424 seconds
5400
Elapsed time: 2784.9377 seconds
5600
Elapsed time: 2956.3059 seconds
5800
Elapsed time: 3071.7924 seconds
6000
Elapsed time: 3274.2199 seconds
6200
Elapsed time: 3413.3732 seconds
6400
Elapsed time: 3569.4521 seconds
6600
Elapsed time: 3723.1193 seconds
6800
Elapsed t

In [7]:
df_eddies.to_pickle(f"/srv/scratch/z5297792/Chapter2/df_eddies_{start_day}_{end_day}.pkl")
df_R.to_pickle(f"/srv/scratch/z5297792/Chapter2/df_R_{start_day}_{end_day}.pkl")
